In [1]:
test = """Valve AA has flow rate=0; tunnels lead to valves DD, II, BB
Valve BB has flow rate=13; tunnels lead to valves CC, AA
Valve CC has flow rate=2; tunnels lead to valves DD, BB
Valve DD has flow rate=20; tunnels lead to valves CC, AA, EE
Valve EE has flow rate=3; tunnels lead to valves FF, DD
Valve FF has flow rate=0; tunnels lead to valves EE, GG
Valve GG has flow rate=0; tunnels lead to valves FF, HH
Valve HH has flow rate=22; tunnel leads to valve GG
Valve II has flow rate=0; tunnels lead to valves AA, JJ
Valve JJ has flow rate=21; tunnel leads to valve II"""
source = test

In [2]:
source = open("./day_16.txt").read()

In [3]:
class Valve:
    valves = {}
    def __init__(self, name, flow_rate, tunnels):
        self.name = name
        self.flow_rate = flow_rate
        self.tunnels = tunnels
        self.valves[name] = self
        self.distances = {}
    def tostring(self):
        return f"{self.name} has flow rate={self.flow_rate}; tunnels lead to valves {' - '.join(self.tunnels)}"
    def paths(self, valve=None, distance=0):        
        if(valve == None):
            for t in self.tunnels:
                self.distances[t] = 1
                self.paths(self.valves[t], distance+1)
        else:
            dist = self.distances.get(valve.name)
            if(dist == None or dist >= distance):
                self.distances[valve.name] = distance;
                for t in [t for t in valve.tunnels if t != self.name ]:
                    self.paths(self.valves[t], distance+1)

In [4]:
for line in source.splitlines():
    words = line.split(" ")
    if words[0] == "Valve":
        name = words[1]
        flow = int(words[4].split("=")[1].split(";")[0])
        tunnels = [w.strip() for w in " ".join(words[9:]).split(", ")]
        Valve(name, flow, tunnels)
for valve in Valve.valves.values(): valve.paths()

In [5]:
goodValves = [v for v in Valve.valves.values() if v.flow_rate > 0]
print([v.name for v in goodValves])

['OQ', 'PD', 'TN', 'SM', 'KX', 'IR', 'BC', 'HR', 'HX', 'BV', 'JQ', 'OF', 'SG', 'FN', 'ZX']


## Part 1: Turning valves solo

In [6]:
turnsleft = 30
position = "AA"
goodValves = [v for v in Valve.valves.values() if v.flow_rate > 0]
class Scores:
    def __init__(self):
        self.maxScore = 0
        self.maxPath = []
        self.paths = []
    def getScore(cls, goodValves, turnsleft, position, score, thisPath = []):
        score += Valve.valves[position].flow_rate * turnsleft
        if(turnsleft < 0): return
        cls.paths.append({"path": thisPath + [position], "score": score})
        if(score > cls.maxScore): 
            cls.maxScore = score
            cls.maxPath = thisPath
        for nextValve in goodValves:
            remainingpaths = [v for v in goodValves if v.name != nextValve.name]
            thisTurnsLeft = turnsleft - Valve.valves[Valve.valves[position].name].distances[nextValve.name]-1
            cls.getScore(remainingpaths, thisTurnsLeft, nextValve.name, score, thisPath + [position])
score = Scores()
score.getScore(goodValves, turnsleft, position, 0)
print(score.maxScore, score.maxPath)

1796 ['AA', 'BC', 'OF', 'OQ', 'TN', 'BV', 'HR', 'PD']


## Part 2: Pachyderm panic!

In [7]:
score = Scores()
score.getScore(goodValves, 26, "AA", 0)
for path in score.paths: path["hash"] = ",".join(path["path"])
bestPaths = []
maxScore = 0
for path in score.paths:
    for path2 in score.paths:
        if len([p for p in path2["path"] if p in path["path"]]) == 1 and path["hash"] < path2["hash"]:
            comboScore = path["score"] + path2["score"]
            if(comboScore > maxScore):
                maxScore = comboScore
                bestPaths = [path["path"], path2["path"]]
maxScore, bestPaths

(1999,
 [['AA', 'BC', 'SG', 'PD', 'FN'],
  ['AA', 'HX', 'OF', 'OQ', 'TN', 'BV', 'HR', 'JQ']])